## Homework

### Dataset

In this homework, we'll build a model for predicting if we have an image of a dog or a cat. For this,
we will use the "Dogs & Cats" dataset that can be downloaded from [Kaggle](https://www.kaggle.com/c/dogs-vs-cats/data). 

You need to download the `train.zip` file.

If you have troubles downloading from Kaggle, use [this link](https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip) instead:

```bash
wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip
```

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

**Note:** You don't need a computer with a GPU for this homework. A laptop or any personal computer should be sufficient. 


### Data Preparation

The dataset contains 12,500 images of cats and 12,500 images of dogs. 

Now we need to split this data into train and validation

* Create a `train` and `validation` folders
* In each folder, create `cats` and `dogs` folders
* Move the first 10,000 images to the train folder (from 0 to 9999) for boths cats and dogs - and put them in respective folders
* Move the remaining 2,500 images to the validation folder (from 10000 to 12499)

You can do this manually or with Python (check `os` and `shutil` packages).


In [46]:
!rm -R train
!rm -R validation

In [47]:
import os 
import shutil

if not os.path.exists('./train.zip'):
  !wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip
  !unzip train.zip

CONTENT_DIR = './'
TRAIN_DIR = CONTENT_DIR + 'train'
VALID_DIR = CONTENT_DIR + 'validation'

# Extract dataset
import zipfile
with zipfile.ZipFile(CONTENT_DIR + 'train.zip', 'r') as zipf:
    zipf.extractall('data')


# split at train and validation folders
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VALID_DIR, exist_ok=True)

# Move the first 10,000 images to the train folder (from 0 to 9999) for boths cats and dogs - and put them in respective folders
files = os.listdir(CONTENT_DIR+'data/train')

for file in files[0:10000]:
  shutil.copy(os.path.join(CONTENT_DIR+'data/train', file), TRAIN_DIR)

for file in files[10000:12500]:
  shutil.copy(os.path.join(CONTENT_DIR+'data/train', file), VALID_DIR)


for folder in [TRAIN_DIR, VALID_DIR]:
  files = os.listdir(folder)
  dog_filenames = [fn for fn in files if fn.startswith('dog')]
  cat_filenames = [fn for fn in files if fn.startswith('cat')]
  make_dirs = [folder + a for a in ['/dog', '/cat']]
  
  for dir, filenames in zip(make_dirs, [dog_filenames, cat_filenames]):
    os.makedirs(dir, exist_ok=True)
    for animal in filenames:
      if os.path.isfile(folder+"/"+animal):
        shutil.move(folder+"/"+animal, dir)

### Model

For this homework we will use Convolutional Neural Network (CNN. Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a covolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`


For clarification about kernel size and max pooling, check [Week #11 Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

In [36]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [37]:
import tensorflow as tf
import tensorflow.keras.layers  as layers
import tensorflow.keras as keras

INPUT_SHAPE=(150,150)

class CNN():
  model : keras.Model = None

  def build(self, input_shape):
    inputs = keras.Input(shape=(*input_shape, 3), name="img")
    
    # backbone
    x = layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units=64, activation='relu')(x)
    
    # detector
    outputs = layers.Dense(units=1, activation='sigmoid')(x)
    self.model = keras.Model(inputs, outputs)

    self.optimizer = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
    self.loss = keras.losses.BinaryCrossentropy(from_logits=False),
    self.model.compile(
        optimizer=self.optimizer, 
        loss=self.loss,
        metrics=["accuracy"])

    return self.model


In [51]:
network = CNN()
model = network.build(input_shape = INPUT_SHAPE)

In [52]:
import numpy as np

input_data = np.random.rand(1, 150, 150, 3)
input_data.shape


(1, 150, 150, 3)

In [53]:
model.predict(input_data)

array([[0.47437868]], dtype=float32)

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`


#### Answer 1) 

We might use keras.losses.BinaryCrossentropy(from_logits=False) with activation='sigmoid' at last layer. 
Other approaches: use SparseCategoricalCrossentropy or CategoricalCrossentropy  but changing the transforming `y` in one-hot encoding




### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

In [47]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img (InputLayer)            [(None, 150, 150, 3)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten_7 (Flatten)         (None, 175232)            0         
                                                                 
 dense_18 (Dense)            (None, 64)                11214912  
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                           

### Generators and Training

For the next two questions, use the following data generator for both train and validation:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and validaition 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)
```

Note `validation_steps=50` - this parameter says "run only 50 steps on the validation data for evaluating the results". 
This way we iterate a bit faster, but don't use the entire validation dataset.
That's why it's important to shuffle the validation dataset as well. 



In [48]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale=1./255
)

train_ds = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=INPUT_SHAPE,
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

val_gen = ImageDataGenerator(
    rescale=1./255
)

val_ds = val_gen.flow_from_directory(
    VALID_DIR,
    target_size=INPUT_SHAPE,
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

Found 10000 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


### Question 3

What is the median of training accuracy for this model?


In [54]:
history = model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 11s 103ms/step - loss: 0.6971 - accuracy: 0.5060 - val_loss: 0.6911 - val_accuracy: 0.5600
Epoch 2/10
100/100 [==============================] - 12s 122ms/step - loss: 0.6896 - accuracy: 0.5450 - val_loss: 0.6880 - val_accuracy: 0.5350
Epoch 3/10
100/100 [==============================] - 12s 122ms/step - loss: 0.6846 - accuracy: 0.5700 - val_loss: 0.6850 - val_accuracy: 0.5560
Epoch 4/10
100/100 [==============================] - 12s 123ms/step - loss: 0.6788 - accuracy: 0.5665 - val_loss: 0.6785 - val_accuracy: 0.5790
Epoch 5/10
100/100 [==============================] - 10s 102ms/step - loss: 0.6768 - accuracy: 0.5660 - val_loss: 0.6728 - val_accuracy: 0.6170
Epoch 6/10
100/100 [==============================] - 10s 102ms/step - loss: 0.6761 - accuracy: 0.5775 - val_loss: 0.6639 - val_accuracy: 0.5980
Epoch 7/10
100/100 [==============================] - 10s 101ms/step - loss: 0.6651 - accuracy: 0.5940 - val_loss: 0.6690 - val_ac

In [59]:
np.mean(history.history['accuracy'])

0.5748999953269959

### Question 4

What is the standard deviation of training loss for this model?


In [62]:
np.std(history.history['loss'])

0.014525656717254233

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`



In [63]:
train_gen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest'
)

train_ds = train_gen.flow_from_directory(
  TRAIN_DIR,
  target_size=INPUT_SHAPE,
  batch_size=20,
  shuffle=True,
  class_mode='binary'
)

Found 10000 images belonging to 2 classes.



### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of validation loss for the model trained with augmentations?


In [64]:
history_2 = model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 20s 199ms/step - loss: 0.6729 - accuracy: 0.5835 - val_loss: 0.6562 - val_accuracy: 0.5930
Epoch 2/10
100/100 [==============================] - 22s 217ms/step - loss: 0.6749 - accuracy: 0.5750 - val_loss: 0.6642 - val_accuracy: 0.5920
Epoch 3/10
100/100 [==============================] - 20s 200ms/step - loss: 0.6691 - accuracy: 0.5785 - val_loss: 0.6509 - val_accuracy: 0.6270
Epoch 4/10
100/100 [==============================] - 22s 219ms/step - loss: 0.6688 - accuracy: 0.5860 - val_loss: 0.6420 - val_accuracy: 0.6170
Epoch 5/10
100/100 [==============================] - 20s 198ms/step - loss: 0.6601 - accuracy: 0.5945 - val_loss: 0.6617 - val_accuracy: 0.5620
Epoch 6/10
100/100 [==============================] - 20s 197ms/step - loss: 0.6632 - accuracy: 0.5865 - val_loss: 0.6496 - val_accuracy: 0.6170
Epoch 7/10
100/100 [==============================] - 22s 218ms/step - loss: 0.6575 - accuracy: 0.6005 - val_loss: 0.6356 - val_ac

In [65]:
np.mean(history_2.history['val_loss'])

0.6473888695240021


### Question 6

What's the average of validation accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?


In [66]:
np.mean(history_2.history['val_accuracy'][-5:])

0.6137999892234802